## 머신러닝 알고리즘 적용 _ 심화
    - 데이터 : 사용자 데이터 + 상품 데이터(과거 상품 구매 이력)
    - 신규 모델 : RandomForest, ExtraTrees, BaggingClassifier, (XGBoost)
    - 업데이트된 데이터 + 기존 모델(DT, LR) 평가척도 
    - 업데이트된 데이터 + 신규 모델 평가척도
    - [+2] 피쳐 엔지니어링
    - [+2] 매개변수 조정
    - 캐글 제출 
    - 머신러닝 파이프라인 흐름도 기록

In [1]:
import pandas as pd
import numpy as np
import pickle
import time
import operator
import xgboost
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import log_loss, f1_score, accuracy_score

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [3]:
# 신규 데이터 로딩

trn = pd.read_csv('../input/train_append_lb_lag.csv').fillna(0)
target = pd.DataFrame(pickle.load(open('../input/target.pkl','rb')), columns=['target'])
tst = pd.read_csv('../input/test_append_lb_lag.csv').fillna(0)
print(trn.shape, target.shape, tst.shape)

((45619, 246), (45619, 1), (929615, 246))


In [4]:
trn.head()

,age,antiguedad,canal_entrada,cod_prov,conyuemp,fecha_alta,ind_actividad_cliente,ind_empleado,ind_nuevo,indext,...,indrel_lag_fiv,indrel_1mes_lag_fiv,indresi_lag_fiv,nomprov_lag_fiv,pais_residencia_lag_fiv,renta_lag_fiv,segmento_lag_fiv,sexo_lag_fiv,tiprel_1mes_lag_fiv,ult_fec_cli_1t_lag_fiv
0,28,34,150,20,2,1012,1,3,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,28,34,150,20,2,1012,1,3,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,37,34,122,20,2,1012,1,3,0,0,...,0.0,0.0,1.0,30.0,36.0,107894.0,1.0,1.0,0.0,-153.0
3,37,34,122,20,2,1012,1,3,0,0,...,0.0,0.0,1.0,30.0,36.0,107894.0,1.0,1.0,0.0,-153.0
4,40,34,122,20,2,1012,1,3,0,0,...,0.0,0.0,1.0,30.0,36.0,93847.0,1.0,0.0,0.0,-153.0


In [5]:
trn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45619 entries, 0 to 45618
Columns: 246 entries, age to ult_fec_cli_1t_lag_fiv
dtypes: float64(225), int64(21)
memory usage: 85.6 MB


In [6]:
tst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929615 entries, 0 to 929614
Columns: 246 entries, age to ult_fec_cli_1t_lag_fiv
dtypes: float64(180), int64(66)
memory usage: 1.7 GB


In [7]:
# 신규 데이터 설명
for col in trn.columns:
    print(col)

age
antiguedad
canal_entrada
cod_prov
conyuemp
fecha_alta
ind_actividad_cliente
ind_empleado
ind_nuevo
indext
indfall
indrel
indrel_1mes
indresi
nomprov
pais_residencia
renta
segmento
sexo
tiprel_1mes
ult_fec_cli_1t
age_lag_one
antiguedad_lag_one
canal_entrada_lag_one
cod_prov_lag_one
conyuemp_lag_one
fecha_alta_lag_one
ind_actividad_cliente_lag_one
ind_ahor_fin_ult1_lag_one
ind_aval_fin_ult1_lag_one
ind_cco_fin_ult1_lag_one
ind_cder_fin_ult1_lag_one
ind_cno_fin_ult1_lag_one
ind_ctju_fin_ult1_lag_one
ind_ctma_fin_ult1_lag_one
ind_ctop_fin_ult1_lag_one
ind_ctpp_fin_ult1_lag_one
ind_deco_fin_ult1_lag_one
ind_dela_fin_ult1_lag_one
ind_deme_fin_ult1_lag_one
ind_ecue_fin_ult1_lag_one
ind_empleado_lag_one
ind_fond_fin_ult1_lag_one
ind_hip_fin_ult1_lag_one
ind_nom_pens_ult1_lag_one
ind_nomina_ult1_lag_one
ind_nuevo_lag_one
ind_plan_fin_ult1_lag_one
ind_pres_fin_ult1_lag_one
ind_reca_fin_ult1_lag_one
ind_recibo_ult1_lag_one
ind_tjcr_fin_ult1_lag_one
ind_valo_fin_ult1_lag_one
ind_viv_fin_ult1_l

In [8]:
# 훈련 데이터와 테스트 데이터 동일 여부 확인
trn.columns == tst.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [9]:
# 빈도가 낮은 타겟은 사전에 제거 (이유: 교차 검증에 활용할 수 없음 + 너무 빈도가 낮아 무의미함)
rem_targets = [2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 17, 18, 19, 21, 22, 23]  # 18 classes
trn = trn[target['target'].isin(rem_targets)]
target = target[target['target'].isin(rem_targets)]
target = LabelEncoder().fit_transform(target)

for t in np.unique(target):
    print(t, sum(target==t))

(0, 9452)
(1, 1934)
(2, 55)
(3, 349)
(4, 222)
(5, 154)
(6, 503)
(7, 33)
(8, 1085)
(9, 1219)
(10, 246)
(11, 21)
(12, 2942)
(13, 4733)
(14, 159)
(15, 5151)
(16, 8218)
(17, 9119)


/Library/Python/2.7/site-packages/sklearn/preprocessing/label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## 평가용 함수 정의

In [10]:
def evaluate(x, y, model):
    trn_scores = dict(); vld_scores = dict()
    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.1, random_state=777)
    for t_ind, v_ind in sss.split(x,y):
        # split data
        x_trn, x_vld = x.iloc[t_ind], x.iloc[v_ind]
        y_trn, y_vld = y[t_ind], y[v_ind]

        # fit model
        model.fit(x_trn, y_trn)
        
        # eval _ trn        
        preds = model.predict_proba(x_trn)

        log_scores = trn_scores.get('log loss', [])
        log_scores.append(log_loss(y_trn, preds))
        trn_scores['log loss'] = log_scores

        # eval _ vld
        preds = model.predict_proba(x_vld)

        log_scores = vld_scores.get('log loss', [])
        log_scores.append(log_loss(y_vld, preds))
        vld_scores['log loss'] = log_scores
    return trn_scores, vld_scores

def print_scores(trn_scores, vld_scores):
    prefix = '        '
    cols = ['log loss']
    print('='*50)
    print('TRAIN EVAL')
    for col in cols:
        print('-'*50)
        print('# {}'.format(col))
        print('# {} Mean : {}'.format(prefix, np.mean(trn_scores[col])))
        print('# {} Raw  : {}'.format(prefix, trn_scores[col]))

    print('='*50)
    print('VALID EVAL')
    for col in cols:
        print('-'*50)
        print('# {}'.format(col))
        print('# {} Mean : {}'.format(prefix, np.mean(vld_scores[col])))
        print('# {} Raw  : {}'.format(prefix, vld_scores[col]))

def print_time(end, start):
    print('='*50)
    elapsed = end - start
    print('{} secs'.format(round(elapsed)))
    
def fit_and_eval(trn, target, model):
    trn_scores, vld_scores = evaluate(trn,target,model)
    print_scores(trn_scores, vld_scores)
    print_time(time.time(), st)    

## 모델 학습 및 평가
    - 모델 종류
        - Decision Tree : 트리 기반 모델
        - Logistic Regression : 선형 모델
        - RandomForest, ExtraTrees : 트리 기반 앙상블 모델
        - BaggingClassifier : 앙상블 모델
        - (XGBoost) : 트리 기반 앙상블 모델
        
    - 훈련/검증 데이터 기반 평가 척도
        - Log Loss
        
    - 검증 데이터 Log Loss 목표 수치 = 1.1

## 피쳐 엔지니어링 (데이터 최적화) [+2]
    - 직접 새로운 변수를 추가 혹은 기존 변수를 삭제하여서 최적의 변수세트 생성해보기
    - 주의: 훈련 데이터에 수행한 변수 변환은 테스트 데이터에도 동일하게 수행해야함
    - 힌트: 금융 상품에 대한 새로운 정보를 넣는 것

In [11]:
cols = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
        'ind_cder_fin_ult1', 'ind_cno_fin_ult1',  'ind_ctju_fin_ult1',
        'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
        'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
        'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
        'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
        'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
        'ind_nomina_ult1',   'ind_nom_pens_ult1', 'ind_recibo_ult1']

print(trn.shape, tst.shape)

# 타겟별 누적 합
lags = ['_lag_one','_lag_two','_lag_thr','_lag_fou','_lag_fiv']
for col in cols:
    trn[col+'_sum'] = trn[[col+lag for lag in lags]].sum(axis=1)
    tst[col+'_sum'] = tst[[col+lag for lag in lags]].sum(axis=1)
    
# 월별 누적 합
for lag in lags:
    trn['sum'+lag] = trn[[col+lag for col in cols]].sum(axis=1)
    tst['sum'+lag] = tst[[col+lag for col in cols]].sum(axis=1)
    
print(trn.shape, tst.shape)

((45595, 246), (929615, 246))
((45595, 275), (929615, 275))


In [12]:
cols = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
        'ind_cder_fin_ult1', 'ind_cno_fin_ult1',  'ind_ctju_fin_ult1',
        'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
        'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
        'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
        'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
        'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
        'ind_nomina_ult1',   'ind_nom_pens_ult1', 'ind_recibo_ult1']
target_cols = [cols[i] for i, col in enumerate(cols) if i in rem_targets]

## 캐글에 직접 결과물 제출하기
    - MAP@7 평가척도를 기반 (https://www.kaggle.com/c/santander-product-recommendation/details/evaluation)
    - 유저당 상위 7개의 제품을 추천해야함

In [ ]:
import xgboost as xgb


# XGB Model 매개변수 정의
num_round = 500
early_stop = 10
xgb_params = {
    'booster': 'gbtree',
    
    # 모델 복잡도
    'max_depth': 2, # 높을 수록 복잡
    #'gamma': 3,    # 낮을 수록 복잡
    #'min_child_weight': 5, # 낮을 수록 복잡

    # 랜덤 샘플링을 통한 정규화
    #'colsample_bylevel': 0.7,
    #'colsample_bytree': 1,
    #'subsample': 0.8,

    # 정규화
    #'reg_alpha': 2,
    #'reg_lambda': 3,

    # 학습 속도
    #'learning_rate': 0.03,
    
    # 기본 설정
    'nthread': 4,
    'num_class': 18,
    'objective': 'multi:softprob',
    'silent': 1,
    'eval_metric': 'mlogloss',
    'seed': 777,
}


# XGBoost 평가용 함수
def evaluate_xgb(x, y):
    trn_scores = dict(); vld_scores = dict()
    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.1, random_state=777)
    for t_ind, v_ind in sss.split(x,y):
        # split data
        x_trn, x_vld = x.iloc[t_ind], x.iloc[v_ind]
        y_trn, y_vld = y[t_ind], y[v_ind]

        dtrn = xgb.DMatrix(x_trn, label=y_trn)
        dvld = xgb.DMatrix(x_vld, label=y_vld)
        watch_list = [(dtrn, 'train'), (dvld, 'eval')]

        # fit xgb
        bst = xgb.train(xgb_params, dtrn, num_round, watch_list, \
                        early_stopping_rounds=early_stop, verbose_eval=True)
        
        # eval _ trn        
        preds = bst.predict(dtrn)

        log_scores = trn_scores.get('log loss', [])
        log_scores.append(log_loss(y_trn, preds))
        trn_scores['log loss'] = log_scores

        # eval _ vld
        preds = bst.predict(dvld)
        
        log_scores = vld_scores.get('log loss', [])
        log_scores.append(log_loss(y_vld, preds))
        vld_scores['log loss'] = log_scores
    return trn_scores, vld_scores

def print_scores(trn_scores, vld_scores):
    prefix = '        '
    cols = ['log loss']
    print('='*50)
    print('TRAIN EVAL')
    for col in cols:
        print('-'*50)
        print('# {}'.format(col))
        print('# {} Mean : {}'.format(prefix, np.mean(trn_scores[col])))
        print('# {} Raw  : {}'.format(prefix, trn_scores[col]))

    print('='*50)
    print('VALID EVAL')
    for col in cols:
        print('-'*50)
        print('# {}'.format(col))
        print('# {} Mean : {}'.format(prefix, np.mean(vld_scores[col])))
        print('# {} Raw  : {}'.format(prefix, vld_scores[col]))

def print_time(end, start):
    print('='*50)
    elapsed = end - start
    print('{} secs'.format(round(elapsed)))
    
def fit_and_eval(trn, target, model):
    trn_scores, vld_scores = evaluate(trn,target,model)
    print_scores(trn_scores, vld_scores)
    print_time(time.time(), st)    

# XGBoost 평가 실행
evaluate_xgb(trn,target)


# XGBoost 기반 결과물 생성 코드
from datetime import datetime
import os

print('='*50)
print('# Test shape : {}'.format(tst.shape))

# 최종 모델 정의 및 학습 실행
dtrn = xgb.DMatrix(trn, label= target)
num_round = 140 # 평가 함수 기반 최적의 num_round 수치 지정
bst = xgb.train(xgb_params, dtrn, num_round, verbose_eval=True)

dtst = xgb.DMatrix(tst)
preds = bst.predict(dtst)
preds = np.fliplr(np.argsort(preds, axis=1))

cols = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
        'ind_cder_fin_ult1', 'ind_cno_fin_ult1',  'ind_ctju_fin_ult1',
        'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
        'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
        'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
        'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
        'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
        'ind_nomina_ult1',   'ind_nom_pens_ult1', 'ind_recibo_ult1']
target_cols = [cols[i] for i, col in enumerate(cols) if i in rem_targets]

final_preds = []
for pred in preds:
    top_products = []
    for i, product in enumerate(pred):
        top_products.append(target_cols[product])
        if i == 6:
            break
    final_preds.append(' '.join(top_products))

temp = pd.read_csv('../input/test_clean.csv')
test_id = temp['ncodpers']
out_df = pd.DataFrame({'ncodpers':test_id, 'added_products':final_preds})
file_name = datetime.now().strftime("result_%Y%m%d%H%M%S") + '.csv'
out_df.to_csv(os.path.join('../output',file_name), index=False)

[0]	train-mlogloss:1.92287	eval-mlogloss:1.92563
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 10 rounds.
[1]	train-mlogloss:1.6923	eval-mlogloss:1.69613
[2]	train-mlogloss:1.54877	eval-mlogloss:1.55233
[3]	train-mlogloss:1.4416	eval-mlogloss:1.4449
[4]	train-mlogloss:1.36517	eval-mlogloss:1.36938
[5]	train-mlogloss:1.30456	eval-mlogloss:1.30978
[6]	train-mlogloss:1.25836	eval-mlogloss:1.26402
[7]	train-mlogloss:1.21343	eval-mlogloss:1.21655
[8]	train-mlogloss:1.18181	eval-mlogloss:1.18392
[9]	train-mlogloss:1.15589	eval-mlogloss:1.15929
[10]	train-mlogloss:1.13339	eval-mlogloss:1.13554
[11]	train-mlogloss:1.11469	eval-mlogloss:1.11696
[12]	train-mlogloss:1.09895	eval-mlogloss:1.10173
[13]	train-mlogloss:1.08592	eval-mlogloss:1.0881
[14]	train-mlogloss:1.07444	eval-mlogloss:1.07763
[15]	train-mlogloss:1.06405	eval-mlogloss:1.06658
[16]	train-mlogloss:1.05526	eval-mlogloss:1.05797
[17]	train-ml

결과물 출력은 https://www.kaggle.com/c/santander-product-recommendation/submissions/attach

## 나만의 머신러닝 파이프라인 흐름도(Flow Chart) 기록하기

- 원천 데이터
    - .

- 전처리
    - .

- 피쳐 엔지니어링 이전 데이터 dimension
    - .

- 피쳐 엔지니어링 
    - .

- 피쳐 엔지니어링 이후 데이터 dimension
    - .

- 모델 튜닝 
    - .

- 검증 결과 
    - .

- 실제 결과 
    - .

예시

- 원천 데이터 
    - Kaggle 경진대회 데이터 train_ver2.csv, test_ver2.csv (Link: https://www.kaggle.com/c/santander-product-recommendation/data)


- 전처리 
    - 결측값을 .fillna 함수를 통해 0으로 대체. (기존 데이터에 0이 존재할 경우 -1로 대체)


- 피쳐 엔지니어링 이전 데이터 dimension:
    - trn : (45619, 246)
    - target : (45619, 1) [18 classes]
    - tst : (929615, 246)


- 피쳐 엔지니어링
    - age_log : log(age + 1)
    - ind..._lag_one : 5월 사용자별 금융상품 보유현황
    - ind..._lag_two : 4월 사용자별 금융상품 보유현황
    - ind..._lag_thr : 3월 사용자별 금융상품 보유현황


- 피쳐 엔지니어링 이후 데이터 dimension:
    - trn : (45619, 250)
    - target : (45619, 1) [18 classes]
    - tst : (929615, 250)


- 모델 튜닝 
    - RandomForest : max_depth = 20 로 복잡도 조정


- 검증 결과 
    - trn logloss : 1.18
    - vld logloss : 1.28


- 실제 결과 
    - Public Leaderboard : 0.025984


## Appendix
    - RandomForest vs ExtraTrees 의 차이란?
        - P. Geurts, D. Ernst., and L. Wehenkel, “Extremely randomized trees”, Machine Learning, 63(1), 3-42, 2006
        - 1) ET의 경우, 변수 샘플링을 boostrap 샘플이 아닌 전체 데이터에서 취한다.
        - 2) ET의 경우, 샘플내 분포에 상관없이 완전한 임의 샘플링으로 데이터 샘플을 취한다.